# 财务舞弊识别 - XGBoost模型训练与评估

本Notebook将使用预处理后的`reduced_data.csv`数据集，训练XGBoost模型进行财务舞弊识别。我们将执行以下步骤：
1. 数据加载与探索
2. 数据分割与预处理
3. 基础XGBoost模型训练
4. 超参数优化
5. 模型评估与可视化
6. 特征重要性分析
7. 结论与建议

In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

## 1. 数据加载与探索

In [4]:
# 加载数据
data = pd.read_csv('reduced_data.csv')
print("数据集形状：", data.shape)
print("前5行数据:")
data.head()

数据集形状： (119060, 31)
前5行数据:


,Stkcd,Accper,Typrep,isviolation,year,quarter,Typrep_encoded,pca_1,pca_2,pca_3,...,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24
0,1,2019-12-31,A,0,2019,4,0,-0.666692,0.204977,-0.049770,...,0.203310,0.079920,0.041992,-0.173091,-0.626385,-0.087111,0.089037,-1.798102,-0.464738,-0.781412
1,1,2019-09-30,A,0,2019,3,0,-0.721589,0.170246,-0.043756,...,0.171783,0.064626,0.043311,-0.144361,-0.511815,-0.065652,0.071402,-1.548745,-0.420389,-0.802067
2,1,2019-06-30,A,0,2019,2,0,-0.795466,0.107475,-0.032212,...,0.108503,0.033621,0.044009,-0.101721,-0.308463,-0.025006,0.040162,-1.126260,-0.325377,-0.823333
3,1,2019-03-31,A,0,2019,1,0,-0.894404,0.026540,-0.017122,...,0.034722,-0.000499,0.039955,-0.023407,-0.063927,0.024758,0.000877,-0.565702,-0.210359,-0.850414
4,1,2018-12-31,A,0,2018,4,0,-0.678292,0.199138,-0.050285,...,0.211462,0.092058,-0.020113,-0.220922,-0.577156,-0.080308,0.087818,-1.782006,-0.456062,-0.795569


In [5]:
# 数据基本信息
print("数据基本信息：")
data.info()

数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119060 entries, 0 to 119059
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Stkcd           119060 non-null  int64  
 1   Accper          119060 non-null  object 
 2   Typrep          119060 non-null  object 
 3   isviolation     119060 non-null  int64  
 4   year            119060 non-null  int64  
 5   quarter         119060 non-null  int64  
 6   Typrep_encoded  119060 non-null  int64  
 7   pca_1           119060 non-null  float64
 8   pca_2           119060 non-null  float64
 9   pca_3           119060 non-null  float64
 10  pca_4           119060 non-null  float64
 11  pca_5           119060 non-null  float64
 12  pca_6           119060 non-null  float64
 13  pca_7           119060 non-null  float64
 14  pca_8           119060 non-null  float64
 15  pca_9           119060 non-null  float64
 16  pca_10          119060 non-null  float64
 17  pc

In [6]:
# 检查缺失值
print("缺失值统计：")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])

缺失值统计：
Series([], dtype: int64)


In [9]:
# 查看类别分布（舞弊 vs 非舞弊）
print("类别分布：")
target_dist = data['isviolation'].value_counts()
print(target_dist)
print("违规样本比例：{:.2f}%".format(target_dist[1] / len(data) * 100))

类别分布：
isviolation
0    101823
1     17237
Name: count, dtype: int64
违规样本比例：14.48%


## 2. 数据分割与预处理

In [10]:
# 特征选择 - 排除非特征列
# 基础信息列：Stkcd（股票代码）、Accper（会计期间）、Typrep（报表类型）不用于模型训练
X = data.drop(['Stkcd', 'Accper', 'Typrep ', 'isviolation'], axis=1)
y = data['isviolation']

print("特征集形状：", X.shape)
print("特征列：", X.columns.tolist())

特征集形状： (119060, 27)
特征列： ['year', 'quarter', 'Typrep_encoded', 'pca_1', 'pca_2', 'pca_3', 'pca_4', 'pca_5', 'pca_6', 'pca_7', 'pca_8', 'pca_9', 'pca_10', 'pca_11', 'pca_12', 'pca_13', 'pca_14', 'pca_15', 'pca_16', 'pca_17', 'pca_18', 'pca_19', 'pca_20', 'pca_21', 'pca_22', 'pca_23', 'pca_24']


In [12]:
# 划分训练集和测试集（80%训练，20%测试）
# 使用StratifiedShuffleSplit保持类别比例
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("训练集形状：", X_train.shape)
print("测试集形状：", X_test.shape)
print("训练集类别分布：")
print(y_train.value_counts())
print("测试集类别分布：")
print(y_test.value_counts())

训练集形状： (95248, 27)
测试集形状： (23812, 27)
训练集类别分布：
isviolation
0    81458
1    13790
Name: count, dtype: int64
测试集类别分布：
isviolation
0    20365
1     3447
Name: count, dtype: int64


In [13]:
# 使用SMOTE处理类别不平衡问题
print("原始训练集类别分布：", Counter(y_train))
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("SMOTE处理后训练集类别分布：", Counter(y_train_resampled))

原始训练集类别分布： Counter({0: 81458, 1: 13790})
SMOTE处理后训练集类别分布： Counter({1: 81458, 0: 81458})


## 3. 基础XGBoost模型训练

In [14]:
# 训练基础XGBoost模型
base_model = xgb.XGBClassifier(
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='auc'
)

# 训练模型
base_model.fit(X_train_resampled, y_train_resampled)

# 在测试集上进行预测
y_pred_base = base_model.predict(X_test)
y_pred_proba_base = base_model.predict_proba(X_test)[:, 1]

# 评估基础模型
print("基础XGBoost模型性能评估：")
print(f"准确率: {accuracy_score(y_test, y_pred_base):.4f}")
print(f"精确率: {precision_score(y_test, y_pred_base):.4f}")
print(f"召回率: {recall_score(y_test, y_pred_base):.4f}")
print(f"F1分数: {f1_score(y_test, y_pred_base):.4f}")
print(f"AUC分数: {roc_auc_score(y_test, y_pred_proba_base):.4f}")

基础XGBoost模型性能评估：
准确率: 0.6941
精确率: 0.2556
召回率: 0.5822
F1分数: 0.3553
AUC分数: 0.7171


In [16]:
# 基础模型混淆矩阵
cm_base = confusion_matrix(y_test, y_pred_base)

# 详细分类报告
print(classification_report(y_test, y_pred_base))

              precision    recall  f1-score   support

           0       0.91      0.71      0.80     20365
           1       0.26      0.58      0.36      3447

    accuracy                           0.69     23812
   macro avg       0.58      0.65      0.58     23812
weighted avg       0.82      0.69      0.74     23812



## 4. 超参数优化

In [17]:
# 定义要搜索的参数空间
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0.1, 1.0, 5.0]
}

In [19]:
# 使用StratifiedKFold进行交叉验证
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 创建XGBoost分类器
xgb_model = xgb.XGBClassifier(
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='auc'
)

# 由于网格搜索计算量较大，这里我们使用较小的参数组合进行演示
# 实际应用中可以增加参数组合以获得更好的性能
param_grid_limited = {
    'n_estimators': [100, 200],
    'max_depth': [5, 7],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9]
}

# 使用GridSearchCV进行超参数搜索
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_limited,
    cv=kfold,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

# 执行搜索
grid_search.fit(X_train_resampled, y_train_resampled)

# 输出最佳参数
print("最佳参数组合：")
print(grid_search.best_params_)
print("最佳交叉验证AUC分数：", grid_search.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
最佳参数组合：
{'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.9}
最佳交叉验证AUC分数： 0.8498268363966887


In [20]:
# 使用最佳参数训练最终模型
best_params = grid_search.best_params_

# 如果GridSearchCV耗时太长，可以使用预设的参数
# 以下是一些经验证的良好参数，可在需要时取消注释使用
# best_params = {
#     'n_estimators': 200,
#     'max_depth': 7,
#     'learning_rate': 0.05,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8
# }

optimized_model = xgb.XGBClassifier(
    **best_params,
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='auc'
)

optimized_model.fit(X_train_resampled, y_train_resampled)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


## 5. 模型评估与可视化

In [22]:
# 在测试集上进行预测
y_pred_opt = optimized_model.predict(X_test)
y_pred_proba_opt = optimized_model.predict_proba(X_test)[:, 1]

# 评估优化后模型
print("优化后XGBoost模型性能评估：")
print(f"准确率: {accuracy_score(y_test, y_pred_opt):.4f}")
print(f"精确率: {precision_score(y_test, y_pred_opt):.4f}")
print(f"召回率: {recall_score(y_test, y_pred_opt):.4f}")
print(f"F1分数: {f1_score(y_test, y_pred_opt):.4f}")
print(f"AUC分数: {roc_auc_score(y_test, y_pred_proba_opt):.4f}")

# 与基础模型对比
print("模型性能对比：")
print(f"基础模型 AUC: {roc_auc_score(y_test, y_pred_proba_base):.4f}")
print(f"优化模型 AUC: {roc_auc_score(y_test, y_pred_proba_opt):.4f}")
print(f"提升幅度: {(roc_auc_score(y_test, y_pred_proba_opt) - roc_auc_score(y_test, y_pred_proba_base)) * 100:.2f}%")

优化后XGBoost模型性能评估：
准确率: 0.7127
精确率: 0.2710
召回率: 0.5825
F1分数: 0.3699
AUC分数: 0.7315
模型性能对比：
基础模型 AUC: 0.7171
优化模型 AUC: 0.7315
提升幅度: 1.43%


In [24]:
# 优化模型混淆矩阵
cm_opt = confusion_matrix(y_test, y_pred_opt)

# 详细分类报告
print(classification_report(y_test, y_pred_opt))

              precision    recall  f1-score   support

           0       0.91      0.73      0.81     20365
           1       0.27      0.58      0.37      3447

    accuracy                           0.71     23812
   macro avg       0.59      0.66      0.59     23812
weighted avg       0.82      0.71      0.75     23812



In [ ]:
# 精确率-召回率曲线
from sklearn.metrics import precision_recall_curve

# 计算精确率-召回率曲线数据
precision_base, recall_base, _ = precision_recall_curve(y_test, y_pred_proba_base)
precision_opt, recall_opt, _ = precision_recall_curve(y_test, y_pred_proba_opt)

# 绘制精确率-召回率曲线
plt.figure(figsize=(10, 8))
plt.plot(recall_base, precision_base, label=f'基础模型')
plt.plot(recall_opt, precision_opt, label=f'优化模型')
plt.xlabel('召回率')
plt.ylabel('精确率')
plt.title('XGBoost模型精确率-召回率曲线')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()

## 7. 模型保存与结论

In [28]:
# 保存优化后的模型
import joblib

# 保存模型
joblib.dump(optimized_model, 'financial_fraud_xgboost_model.pkl')
print('模型已保存为 financial_fraud_xgboost_model.pkl')

# 保存最佳参数
import json
with open('best_params.json', 'w') as f:
    json.dump(best_params, f)
print('最佳参数已保存为 best_params.json')

模型已保存为 financial_fraud_xgboost_model.pkl
最佳参数已保存为 best_params.json


## 附录：模型进一步调优建议

In [ ]:
# 模型进一步优化的可能方向
print('''
模型进一步优化的可能方向：

1. 更多的超参数搜索：
   - 扩大参数网格范围
   - 使用RandomizedSearchCV进行更广泛的搜索
   - 考虑使用贝叶斯优化方法

2. 特征工程：
   - 尝试创建更多财务比率特征
   - 考虑时间序列特征（如趋势、季节性等）
   - 探索特征交互项

3. 集成学习：
   - 尝试XGBoost与其他模型（如LightGBM、CatBoost）的集成
   - 考虑Stacking或Voting集成方法

4. 不平衡数据处理：
   - 尝试不同的过采样/欠采样策略
   - 调整类别权重
   - 使用自定义损失函数

5. 模型解释性增强：
   - 使用SHAP值进行更深入的特征解释
   - 构建部分依赖图理解特征影响

6. 业务规则整合：
   - 将领域专家知识转化为规则融入模型
   - 考虑混合推理系统''')